# Parse JDs, extract skills

But lowercase the data first

In [40]:
import os
import re
import json
from pprint import pprint
from bs4 import BeautifulSoup
from bs4.element import Comment

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import gensim
from pprint import pprint

from stop_words import get_stop_words

import codecs
import os
import time

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')

import spacy
import pandas as pd
import itertools as it
import en_core_web_sm

import spacy
nlp = spacy.load('en')



[nltk_data] Downloading package punkt to /home/goodgame/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/goodgame/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/goodgame/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape

/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape

In [5]:
import codecs
import boto3
# from collections import defaultdict
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [10]:
s3 = boto3.resource('s3')

In [11]:
# Raghu's code to pull down jobs
BUCKET_NAME = 'tech-salary-project'

In [12]:
%%time

bucket = s3.Bucket(BUCKET_NAME)
all_job_titles = []
all_job_ids = []
all_summaries = []
for o in bucket.objects.all():
    if o.key.startswith('salaries'):
        continue
    
    object = bucket.Object(o.key)
    try :
        lines = object.get()['Body'].read().decode('utf-8').splitlines()
        for line in lines:
            d = json.loads(line)
            
            title = d['title']
            jid = d['id']
            summary = d['summary']
            
            all_job_ids.append(jid)
            all_job_titles.append(title)
            all_summaries.append(summary)
    except Exception as e:
        continue

CPU times: user 29 s, sys: 1.26 s, total: 30.2 s
Wall time: 10min 48s


In [13]:
print('Number of all job titles:', len(all_job_titles))
uniq_job_titles = set(all_job_titles)
print('Number of unique job titles:', len(uniq_job_titles))

print('Number of all job ids:', len(all_job_ids))
uniq_job_ids = set(all_job_ids)
print('Number of unique job ids:', len(uniq_job_ids))

uniq_job_ids = set(all_job_ids)
print('Number of unique job ids:', len(uniq_job_ids))

print('\nNumber of summaries:',len(all_summaries))
unique_summaries = set(all_summaries)
print('Number of unique summaries:', len(unique_summaries))

Number of all job titles: 41777
Number of unique job titles: 21917
Number of all job ids: 41777
Number of unique job ids: 39692
Number of unique job ids: 39692

Number of summaries: 41777
Number of unique summaries: 39300


## Use BS4 to convert the 'summary' field HTML into visible text, and load the files into memory

In [14]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def make_text(text):
    soup = BeautifulSoup(text, "html5lib")
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip().lower() for t in visible_texts) # Modified 27MAR18 to produce lowercase results

In [15]:
%%time
texts = []

for item in unique_summaries:
    texts.append(make_text(item))

CPU times: user 4min 48s, sys: 248 ms, total: 4min 48s
Wall time: 4min 48s


In [17]:
# write to disk as a single large file
with open('../models_data/s3_visible_texts_28mar.txt', 'w') as outfile:
    for item in texts:
        outfile.write(item)
        outfile.write('\n')

# Note: All texts are lowercase at this point.

In [143]:
# Optional preprocessing step... maybe see if this works later

# remove words that appear only once
# texts = [[word for word in document.lower().split() if word not in stop_words]
#          for document in parsed_dataset]

# from collections import defaultdict
# frequency = defaultdict(int)
# for text in texts:
#     for token in text:
#         frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [18]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    for review in filename:
        yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [19]:
# %%time

# with codecs.open('../models_data_lower/spacy_jds_concat_parsed.txt', 'w', encoding='utf_8') as f:
#     for sentence in lemmatized_sentence_corpus(texts): # Changed 28MAR18 to include tokenization function call
#         f.write(sentence + '\n')

In [20]:
unigram_sentences = LineSentence('../models_data_lower/spacy_jds_concat_parsed.txt')

# Stopped Here 18 1312 MAR 18

In [21]:
# print some examples
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

-PRON- relish the opportunity to work on build optimization that improve datum processing performance across different use case that include batch request response and real time system

-PRON- responsibility the successful candidate will be base in redwood city ca and will be responsible for facilitate the collaboration and engagement between senior member of the team to ensure sound design and implementation of big functional area for the product incubate and design the next generation platform for complex heterogeneous system and application involve cut edge technology work closely with architect and development and qa engineer to discuss and evangelize feature collaborate with geographically disperse cross functional team in an agile environment participate in internal cross team meeting scoping decision make and technical documentation mentor junior engineer on technical architectural design and related issue proactively identify architectural weakness and recommend appropriate sol

In [22]:
%%time

bigram_model = Phrases(unigram_sentences)
bigram_model.save('../models_data_lower/spacy_bigram_model_all_PARSED')

CPU times: user 38.4 s, sys: 648 ms, total: 39.1 s
Wall time: 39.1 s


In [23]:
# load the finished model from disk
bigram_model = Phrases.load('../models_data_lower/spacy_bigram_model_all_PARSED')

In [24]:
%%time

with codecs.open('../models_data_lower/spacy_bigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 25s, sys: 468 ms, total: 1min 26s
Wall time: 1min 26s


In [25]:
bigram_sentences = LineSentence('../models_data_lower/spacy_bigram_sentences_PARSED.txt')

In [50]:
# # print examples; certain bigrams are underlined
# for bigram_sentence in it.islice(bigram_sentences, 240, 250):
#     print(u' '.join(bigram_sentence))
#     print(u'')

this associate product marketing manager be an important member of the workfront team with key responsibility include the assistance of development of product lead go to market strategy include launch and team up on the execution of the product marketing plan in collaboration with the marketing sale and customer experience team

in this role -PRON- will lead go to market strategy and plan that leverage product positioning packaging and pricing to attain product business goal

-PRON- will develop product messaging guide product demo development create buyer_persona and understand the buying process and enable the business with market and competitive intelligence for -PRON- product

-PRON- will also assist in ensure successful product launch through the product launch process in collaboration with the launch manager and representative from engineering product management solution marketing demand generation sale education and customer success team

-PRON- be drive by lead and serve in gre

In [27]:
%%time
trigram_model = Phrases(bigram_sentences)
trigram_model.save('../models_data_lower/spacy_trigram_model_all_PARSED')

CPU times: user 38.5 s, sys: 638 ms, total: 39.2 s
Wall time: 38.9 s


In [28]:
# load the finished model from disk
trigram_model = Phrases.load('../models_data_lower/spacy_trigram_model_all_PARSED')

In [29]:
%%time

with codecs.open('../models_data_lower/spacy_trigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 21s, sys: 360 ms, total: 1min 21s
Wall time: 1min 21s


In [30]:
trigram_sentences = LineSentence('../models_data_lower/spacy_trigram_sentences_PARSED.txt')

In [31]:
for trigram_sentence in it.islice(trigram_sentences, 240, 250):
    print(u' '.join(trigram_sentence))
    print(u'')

this associate product marketing manager be an important member of the workfront team with key responsibility include the assistance of development of product lead go to market strategy include launch and team up on the execution of the product marketing plan in collaboration with the marketing sale and customer experience team

in this_role -PRON- will lead go to market strategy and plan that leverage product positioning_packaging and pricing to attain product business goal

-PRON- will develop product messaging guide product demo development create buyer_persona and understand the buying process and enable the business with market and competitive_intelligence for -PRON- product

-PRON- will also assist in ensure successful product launch through the product launch process in collaboration with the launch manager and representative from engineering product management solution marketing demand_generation sale education and customer success team

-PRON- be drive by lead and serve in gre

In [43]:
# NLTK stopwords format is not a list; use this version
stopwords = get_stop_words('english')
print(type(stopwords))

<class 'list'>


#### Final preprocessing

In [45]:
%%time

with codecs.open('../models_data_lower/spacy_trigram_transformed_jds_all_PARSED.txt', 'w', encoding='utf_8') as f:
    for parsed_review in nlp.pipe(line_review(texts),
                                  batch_size=10000, n_threads=16):
        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in stopwords]

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 4h 28min 52s, sys: 5h 52min 44s, total: 10h 21min 37s
Wall time: 1h 9min 55s


In [46]:
%%time

trigram_reviews = LineSentence('../models_data_lower/spacy_trigram_sentences_PARSED.txt')

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
trigram_dictionary.compactify()

trigram_dictionary.save('../models_data_lower/spacy_trigram_dict_all.dict')
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load('../models_data_lower/spacy_trigram_dict_all.dict')

CPU times: user 25 s, sys: 267 ms, total: 25.3 s
Wall time: 25.2 s


In [47]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [48]:
%%time

# generate bag-of-words representations for all JDs and save them as a matrix
MmCorpus.serialize('../models_data_lower/spacy_trigram_bow_corpus_all.mm',
                   trigram_bow_generator('../models_data_lower/spacy_trigram_sentences_PARSED.txt'))

CPU times: user 37 s, sys: 979 ms, total: 37.9 s
Wall time: 38 s
